In [1]:
import re
import os
from openpyxl import load_workbook

rootdir = "C:/Users/kj34/Desktop/swc-python/2. Stuart Automation Project/3. Prototypes/compound_data"

collected_data = {
    
    "compound_name_dict" : [],
    "scf_parent_dict" : [],
    "zero_point_dict" : [],
    "free_energy_dict" : [],
    "bjd_dict" : [],
    "benzene_dict" : [],
    "bs2_dict" : []
    
}

def Bjd(fin, file_data):
    try:
        if "bjd" in fin.name:
            bjdresult = re.search("SCF Done:\s+\S+\s=\s*(-?[0-9.]+)", file_data)
            collected_data["bjd_dict"].append(bjdresult.group(1))
    except:
        pass

def Solvent(fin, file_data):
    try:
        if "benzene" in fin.name:
            benzene_result = re.search("SCF Done:\s+\S+\s=\s*(-?[0-9.]+)", file_data)
            collected_data["benzene_dict"].append(benzene_result.group(1))
    except:
        pass
            
def Bs2(fin, file_data):
    try:
        if "bs2" in fin.name:
            bs2_result = re.search("SCF Done:\s+\S+\s=\s*(-?[0-9.]+)", file_data)
            collected_data["bs2_dict"].append(bs2_result.group(1))
    except:
        pass
        
        
def Parent(fin, file_data):      
    try:
        if "bjd" not in fin.name and "benzene" not in fin.name and "bs2" not in fin.name:
            parent_result = list(re.findall("SCF Done:\s+\S+\s=\s*(-?[0-9.]*)", file_data))
            scf_parent = parent_result[-1]
            collected_data["scf_parent_dict"].append(scf_parent)
            zero_point_energies = re.search("zero-point Energies=\s+(-?[0-9.]+)", file_data)
            collected_data["zero_point_dict"].append(zero_point_energies.group(1))
            thermal_free_energies = re.search("thermal Free Energies=\s+(-?[0-9.]+)", file_data)
            collected_data["free_energy_dict"].append(thermal_free_energies.group(1))
    except:
        pass
        

for roots, dir, files in os.walk(rootdir):          
    
    for dirs in dir:
        collected_data["compound_name_dict"].append(dirs)
    
    for file in files:
        with open (os.path.join(roots, file), "r") as fin:    

            file_data = fin.read()
            
            Bjd(fin, file_data)
            Solvent(fin, file_data)
            Bs2(fin, file_data)
            Parent(fin, file_data)
                    

A_columns = ["A2", "A3", "A4", "A5"]
B_columns = ["B2", "B3", "B4", "B5"]
C_columns = ["C2", "C3", "C4", "C5"]
D_columns = ["D2", "D3", "D4", "D5"]                
E_columns = ["E2", "E3", "E4", "E5"]                
F_columns = ["F2", "F3", "F4", "F5"]
G_columns = ["G2", "G3", "G4", "G5"]                

spreadsheetfile = "data.xlsx"
        
wb = load_workbook(spreadsheetfile)      
ws = wb.worksheets[0]        

for i in range(0, 4):
    ws[A_columns[i]] = (collected_data["compound_name_dict"][i])

for i in range(0, 4):
    ws[B_columns[i]] = (collected_data["scf_parent_dict"][i])

for i in range(0, 4):
    ws[C_columns[i]] = (collected_data["zero_point_dict"][i])

for i in range(0, 4):
    ws[D_columns[i]] = (collected_data["free_energy_dict"][i])

for i in range(0, 4):
    ws[E_columns[i]] = (collected_data["benzene_dict"][i])

for i in range(0, 4):
    ws[F_columns[i]] = (collected_data["bjd_dict"][i])
    
for i in range(0, 4):
    ws[G_columns[i]] = (collected_data["bs2_dict"][i])

wb.save(spreadsheetfile)
